In [1]:
#dependencies
import pandas as pd
from sqlalchemy import create_engine

starbucks_path = "Resources/us_starbucks_location.csv"
weather_path = "Resources/weather_data_from_starbucks.csv"

In [2]:
#read the starbucks csv and fixed the column name
starbucks_df = pd.read_csv(starbucks_path)
del starbucks_df["Unnamed: 0"]
starbucks_df = starbucks_df.rename(columns={"Store Number":"store_number","Store Name":"store_name","Ownership Type":"ownership_type",
                                            "Street Address":"street_address","City":"city", "State/Province": "state_province", "Postcode": "postcode", "Phone Number": "phone_number"})

starbucks_df.head()

,store_number,store_name,ownership_type,street_address,city,state_province,postcode,phone_number
0,3513-125945,Safeway-Anchorage #1809,Licensed,5600 Debarr Rd Ste 9,Anchorage,AK,995042300,907-339-0900
1,74352-84449,Safeway-Anchorage #2628,Licensed,1725 Abbott Rd,Anchorage,AK,995073444,907-339-2800
2,12449-152385,Safeway - Anchorage #1813,Licensed,1501 Huffman Rd,Anchorage,AK,995153596,907-339-1300
3,24936-233524,100th & C St - Anchorage,Company Owned,"320 W. 100th Ave, 100, Southgate Shopping Ctr ...",Anchorage,AK,99515,(907) 227-9631
4,8973-85630,Old Seward & Diamond,Company Owned,1005 E Dimond Blvd,Anchorage,AK,995152050,907-344-4160


In [3]:
#read the weather csv
weather_df = pd.read_csv(weather_path)
del weather_df["Unnamed: 0"]

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Anchorage,61.22,-149.90,-6.0,26,1,4.18,US,1604370751
1,Eagle River,45.92,-89.24,5.0,56,1,2.10,US,1604370712
2,Fairbanks,64.84,-147.72,-25.0,75,20,0.59,US,1604370756
3,Homer,59.64,-151.55,-3.0,23,75,2.10,US,1604370929
4,Juneau,58.30,-134.42,-2.0,92,90,0.92,US,1604370647


In [4]:
us_weather_df = weather_df[weather_df["Country"].isin(["US"])]
us_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Anchorage,61.22,-149.90,-6.0,26,1,4.18,US,1604370751
1,Eagle River,45.92,-89.24,5.0,56,1,2.10,US,1604370712
2,Fairbanks,64.84,-147.72,-25.0,75,20,0.59,US,1604370756
3,Homer,59.64,-151.55,-3.0,23,75,2.10,US,1604370929
4,Juneau,58.30,-134.42,-2.0,92,90,0.92,US,1604370647


In [5]:
#renamed the column names and put into a new DF
new_weather_df = us_weather_df[["City", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]].copy()
new_weather_df = new_weather_df.rename(columns ={"City": "city", "Max Temp": "max_temp", "Humidity": "humidity", 
                                                 "Cloudiness": "cloudiness", "Wind Speed": "wind_speed", "Date": "date"})
new_weather_df.head()

,city,max_temp,humidity,cloudiness,wind_speed,date
0,Anchorage,-6.0,26,1,4.18,1604370751
1,Eagle River,5.0,56,1,2.10,1604370712
2,Fairbanks,-25.0,75,20,0.59,1604370756
3,Homer,-3.0,23,75,2.10,1604370929
4,Juneau,-2.0,92,90,0.92,1604370647


In [6]:
#merged 2 DFs
merged_df = pd.merge(starbucks_df, new_weather_df,on="city",how='inner')
merged_df.head()

,store_number,store_name,ownership_type,street_address,city,state_province,postcode,phone_number,max_temp,humidity,cloudiness,wind_speed,date
0,3513-125945,Safeway-Anchorage #1809,Licensed,5600 Debarr Rd Ste 9,Anchorage,AK,995042300,907-339-0900,-6.0,26,1,4.18,1604370751
1,74352-84449,Safeway-Anchorage #2628,Licensed,1725 Abbott Rd,Anchorage,AK,995073444,907-339-2800,-6.0,26,1,4.18,1604370751
2,12449-152385,Safeway - Anchorage #1813,Licensed,1501 Huffman Rd,Anchorage,AK,995153596,907-339-1300,-6.0,26,1,4.18,1604370751
3,24936-233524,100th & C St - Anchorage,Company Owned,"320 W. 100th Ave, 100, Southgate Shopping Ctr ...",Anchorage,AK,99515,(907) 227-9631,-6.0,26,1,4.18,1604370751
4,8973-85630,Old Seward & Diamond,Company Owned,1005 E Dimond Blvd,Anchorage,AK,995152050,907-344-4160,-6.0,26,1,4.18,1604370751


In [10]:
#link to Postgres and need to enter your own password
rds_connection_string = "postgres:<Enter your password>@localhost:5432/starbucks_weather_db"

In [11]:
#start the engine
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
#find the table name
engine.table_names()

['weather_data', 'starbucks_location']

In [14]:
#put weather df into the weather_data table
new_weather_df.to_sql(name = 'weather_data',  con=engine, if_exists='append', index=False)

In [15]:
#put starbucks df into starbucks_location table
starbucks_df.to_sql(name = 'starbucks_location',  con=engine, if_exists='append', index=False)

In [16]:
#see if the data is in the weather_data table
pd.read_sql_query('select * from weather_data', con=engine).head()

,city,max_temp,humidity,cloudiness,wind_speed,date
0,Anchorage,-6,26,1,4,1604370751
1,Eagle River,5,56,1,2,1604370712
2,Fairbanks,-25,75,20,1,1604370756
3,Homer,-3,23,75,2,1604370929
4,Juneau,-2,92,90,1,1604370647


In [17]:
#see if the data is in the starbucks_location table
pd.read_sql_query('select * from starbucks_location', con=engine).head()

,store_number,store_name,ownership_type,street_address,city,state_province,postcode,phone_number
0,3513-125945,Safeway-Anchorage #1809,Licensed,5600 Debarr Rd Ste 9,Anchorage,AK,995042300,907-339-0900
1,74352-84449,Safeway-Anchorage #2628,Licensed,1725 Abbott Rd,Anchorage,AK,995073444,907-339-2800
2,12449-152385,Safeway - Anchorage #1813,Licensed,1501 Huffman Rd,Anchorage,AK,995153596,907-339-1300
3,24936-233524,100th & C St - Anchorage,Company Owned,"320 W. 100th Ave, 100, Southgate Shopping Ctr ...",Anchorage,AK,99515,(907) 227-9631
4,8973-85630,Old Seward & Diamond,Company Owned,1005 E Dimond Blvd,Anchorage,AK,995152050,907-344-4160
